In [36]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime


import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.set_option('display.max_colwidth', None)  # Display full content of each column
pd.set_option('display.max_columns', None)   # Display all columns
pd.set_option('display.width', 5000)         # Set display width

plt.style.use('ggplot')
plt.style.use('fivethirtyeight')


In [37]:
df = pd.read_csv("Features.csv")  # Update with your file path
df.columns

Index(['Transaction_Date', 'Description', 'Reference_No./Cheque_No.', 'Debit', 'Credit', 'Balance', 'Transaction_Type', 'Transaction_Mode', 'DR/CR_Indicator', 'Transaction_ID', 'Recipient_Name', 'Bank', 'UPI_ID', 'Note', 'dayofweek', 'weekday', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear', 'weekofmonth', 'date_offset', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end', 'transaction_amount', 'is_large_transaction', 'transaction_count_per_day', 'transaction_count_per_week', 'transaction_count_per_month', 'average_transaction_per_day', 'cumulative_spent_per_day', 'time_since_last_transaction', 'rolling_avg_transaction_7d', 'rolling_avg_transaction_30d'], dtype='object')

In [38]:
import dtale
# dtale.show(df)

In [42]:
print("Start date:", df['Transaction_Date'].min())
print("End date:", df['Transaction_Date'].max())

print("Min transaction amount:", df['transaction_amount'].min())
print("Max transaction amount:", df['transaction_amount'].max())

Start date: 2022-05-31
End date: 2024-11-06
Min transaction amount: 0.0
Max transaction amount: 99069.8


In [43]:
missing_values = df.isnull().sum()
print("Missing values in each column:")
missing_values[missing_values > 0]


Missing values in each column:


Reference_No./Cheque_No.     21
Transaction_Mode             10
Transaction_ID               34
Recipient_Name               34
Bank                        111
UPI_ID                       34
Note                         52
dtype: int64

In [44]:
numerical_cols = df.select_dtypes(include=[np.number]).columns
numerical_summary = df[numerical_cols].describe()
numerical_summary


,Debit,Credit,Balance,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,weekofmonth,date_offset,is_weekend,is_month_start,is_month_end,is_quarter_start,is_quarter_end,transaction_amount,is_large_transaction,transaction_count_per_day,transaction_count_per_week,transaction_count_per_month,average_transaction_per_day,cumulative_spent_per_day,time_since_last_transaction,rolling_avg_transaction_7d,rolling_avg_transaction_30d
count,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000,741.000000
mean,295.891903,296.845412,1619.971835,2.916329,2.601889,6.811066,2023.311741,191.715250,15.512821,27.730094,2.681511,480.128205,0.271255,0.031039,0.031039,0.008097,0.012146,295.891903,0.024291,3.229420,18.827260,75.058030,295.891903,774.271930,28.825911,295.554520,293.494796
std,3657.558653,3698.509961,3876.740810,2.009038,0.959700,2.934775,0.631373,89.065637,8.606497,12.700608,1.242728,330.018265,0.444908,0.173541,0.173541,0.089680,0.109610,3657.558653,0.154057,2.574038,8.587537,30.762001,1820.737977,6340.792462,111.852162,1383.053330,661.262341
min,0.000000,0.000000,12.790000,0.000000,1.000000,1.000000,2022.000000,9.000000,1.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,798.920000,1.000000,2.000000,4.000000,2023.000000,120.000000,8.000000,18.000000,2.000000,197.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,12.000000,50.000000,48.000000,35.000000,0.000000,73.874286,99.133333
50%,56.000000,0.000000,1214.820000,3.000000,2.000000,6.000000,2023.000000,172.000000,16.000000,25.000000,3.000000,405.000000,0.000000,0.000000,0.000000,0.000000,0.000000,56.000000,0.000000,2.000000,18.000000,75.000000,100.000000,142.000000,0.000000,115.142857,138.691333
75%,167.000000,10.000000,1753.820000,5.000000,3.000000,9.000000,2024.000000,269.000000,23.000000,39.000000,4.000000,703.000000,1.000000,0.000000,0.000000,0.000000,0.000000,167.000000,0.000000,4.000000,25.000000,107.000000,194.666667,339.000000,24.000000,190.000000,211.316667
max,99069.800000,100000.000000,101359.590000,6.000000,4.000000,12.000000,2024.000000,365.000000,31.000000,52.000000,5.000000,1299.000000,1.000000,1.000000,1.000000,1.000000,1.000000,99069.800000,1.000000,13.000000,37.000000,117.000000,24839.700000,99358.800000,2400.000000,14420.971429,3516.950667


In [45]:
from prettytable import PrettyTable
# Detecting Outliers using IQR method
def detect_outliers(col):
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = col[(col < lower_bound) | (col > upper_bound)]
    return outliers

outlier_dict = {}
for col in numerical_cols:
    outliers = detect_outliers(df[col])
    outlier_dict[col] = len(outliers)



table = PrettyTable()
table.field_names = ["Numerical Column", "Number of Outliers"]

for col, outlier_count in outlier_dict.items():
    table.add_row([col, outlier_count])

print(table)


+-----------------------------+--------------------+
|       Numerical Column      | Number of Outliers |
+-----------------------------+--------------------+
|            Debit            |         71         |
|            Credit           |        171         |
|           Balance           |         51         |
|          dayofweek          |         0          |
|           quarter           |         0          |
|            month            |         0          |
|             year            |         0          |
|          dayofyear          |         0          |
|          dayofmonth         |         0          |
|          weekofyear         |         0          |
|         weekofmonth         |         0          |
|         date_offset         |         0          |
|          is_weekend         |         0          |
|        is_month_start       |         23         |
|         is_month_end        |         23         |
|       is_quarter_start      |         6     

In [46]:

# Categorical column analysis
categorical_cols = df.select_dtypes(include=['object']).columns
distinct_counts = {col: df[col].nunique() for col in categorical_cols}
most_frequent = {col: df[col].value_counts().idxmax() for col in categorical_cols}
from prettytable import PrettyTable

# Create a table for the categorical column analysis
table = PrettyTable()

# Set the table field names
table.field_names = ["Categorical Column", "Distinct Values", "Most Frequent Value"]

# Add rows to the table
for col in categorical_cols:
    table.add_row([col, distinct_counts[col], most_frequent[col]])

# Display the table
print(table)


+--------------------------+-----------------+--------------------------+
|    Categorical Column    | Distinct Values |   Most Frequent Value    |
+--------------------------+-----------------+--------------------------+
|     Transaction_Date     |       360       |        2024-06-16        |
|       Description        |       731       |    CREDIT INTEREST--     |
| Reference_No./Cheque_No. |       224       | TRANSFER TO4897692162094 |
|     Transaction_Type     |        2        |            TO            |
|     Transaction_Mode     |        5        |           UPI            |
|     DR/CR_Indicator      |        2        |            DR            |
|      Transaction_ID      |       639       |          XX8237          |
|      Recipient_Name      |       258       |        9890160567        |
|           Bank           |        28       |           YESB           |
|          UPI_ID          |       268       |        paytmqr281        |
|           Note           |        64

In [47]:
# Correlation matrix of numerical features
correlation_matrix = df[numerical_cols].corr()
correlation_matrix


,Debit,Credit,Balance,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,weekofmonth,date_offset,is_weekend,is_month_start,is_month_end,is_quarter_start,is_quarter_end,transaction_amount,is_large_transaction,transaction_count_per_day,transaction_count_per_week,transaction_count_per_month,average_transaction_per_day,cumulative_spent_per_day,time_since_last_transaction,rolling_avg_transaction_7d,rolling_avg_transaction_30d
Debit,1.000000,-0.006502,0.007096,0.055722,0.013009,-0.002454,0.047788,-0.002979,-0.005976,-0.003904,-0.018146,-0.012496,0.062050,-0.009480,-0.005549,-0.006115,0.000799,1.000000,0.305561,0.026527,-0.002171,-0.047327,0.497801,0.575287,-0.011332,0.378594,0.178840
Credit,-0.006502,1.000000,0.956587,0.057440,0.013648,-0.001385,0.045730,-0.002663,-0.013850,-0.003641,-0.025685,-0.012796,0.062077,-0.004795,-0.005938,0.009040,0.002395,-0.006502,-0.012673,0.021904,-0.003336,-0.042632,0.491086,-0.008129,-0.001437,-0.004736,-0.004329
Balance,0.007096,0.956587,1.000000,0.035866,-0.024782,-0.059680,0.059479,-0.063912,-0.043084,-0.064416,-0.047939,-0.067985,0.042950,-0.022197,-0.026081,-0.004462,-0.009706,0.007096,0.027685,0.119363,0.052843,0.025700,0.492191,0.011925,-0.009206,0.007215,-0.015360
dayofweek,0.055722,0.057440,0.035866,1.000000,-0.003983,0.012213,-0.020958,0.013257,0.008346,-0.009466,-0.013935,0.071852,0.788913,0.011335,0.011335,-0.003735,0.127353,0.055722,0.045871,0.092303,-0.006557,-0.027363,0.111935,0.096263,0.007428,0.069453,0.022357
quarter,0.013009,0.013648,-0.024782,-0.003983,1.000000,0.959722,-0.200801,0.956769,-0.074723,0.957170,-0.075864,0.327279,-0.018925,0.025612,-0.023072,0.037505,-0.056743,0.013009,0.038078,-0.099190,-0.186591,-0.219893,0.026133,0.025176,0.027893,0.036332,0.057089
month,-0.002454,-0.001385,-0.059680,0.012213,0.959722,1.000000,-0.245305,0.995349,-0.092248,0.995137,-0.093220,0.364752,-0.000025,0.032757,-0.022963,0.005820,-0.018062,-0.002454,0.010165,-0.099798,-0.207411,-0.256169,-0.004930,-0.001194,0.021059,-0.004291,0.020169
year,0.047788,0.045730,0.059479,-0.020958,-0.200801,-0.245305,1.000000,-0.253100,-0.109040,-0.245984,-0.100632,-0.376060,-0.007985,0.047236,-0.002097,0.026958,0.003795,0.047788,0.005400,0.219522,0.313267,0.337491,0.095998,0.085744,-0.079657,0.126802,0.273539
dayofyear,-0.002979,-0.002663,-0.063912,0.013257,0.956769,0.995349,-0.253100,1.000000,0.003844,0.999718,0.000510,0.367644,-0.000708,0.003720,0.007130,-0.009016,0.000355,-0.002979,0.012619,-0.101518,-0.211216,-0.259146,-0.005983,-0.001615,0.018148,-0.001078,0.009809
dayofmonth,-0.005976,-0.013850,-0.043084,0.008346,-0.074723,-0.092248,-0.109040,0.003844,1.000000,0.002888,0.975025,-0.003540,-0.007085,-0.302009,0.313238,-0.152458,0.191071,-0.005976,0.025245,-0.023801,-0.028950,-0.013516,-0.012004,-0.005667,-0.031784,0.031767,-0.119821
weekofyear,-0.003904,-0.003641,-0.064416,-0.009466,0.957170,0.995137,-0.245984,0.999718,0.002888,1.000000,0.000112,0.364017,-0.018594,0.003806,0.006872,-0.008757,-0.002496,-0.003904,0.011643,-0.102229,-0.209226,-0.256602,-0.007842,-0.003184,0.017447,-0.001749,0.011264


In [1]:
# import pandas as pd
# import plotly.express as px
# import dash
# from dash import dcc, html, Input, Output, State
# import dash_bootstrap_components as dbc
# from datetime import datetime

# # ------------------------
# # Data Loading and Preprocessing
# # ------------------------
# df = pd.read_csv("Merge_Proccessed.csv")  # Replace with your actual data source

# # Convert Transaction_Date to datetime and extract components for breakdown analysis
# df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'])
# df['Year'] = df['Transaction_Date'].dt.year
# df['Month_Num'] = df['Transaction_Date'].dt.month
# df['Month'] = df['Transaction_Date'].dt.strftime('%B')  # e.g., January, February, etc.
# df['Week_Num'] = df['Transaction_Date'].dt.strftime('%U').astype(int) + 1  # Week number starting at 1
# df['Day'] = df['Transaction_Date'].dt.date

# # ------------------------
# # Helper Functions for Breakdown Analysis
# # ------------------------
# def generate_summary(df_filtered, group_col):
#     """
#     Group the filtered DataFrame by a column and summarize Debit, Credit, and Balance.
#     Also count the total transactions.
#     """
#     df_grouped = df_filtered.groupby(group_col).agg({
#         'Debit': 'sum', 
#         'Credit': 'sum', 
#         'Balance': 'last'
#     }).reset_index()

#     transaction_counts = df_filtered.groupby(group_col).size().reset_index(name='Total_Transactions')
#     df_summary = df_grouped.merge(transaction_counts, on=group_col)
#     # Recalculate balance as income minus spending
#     df_summary['Balance'] = df_summary['Credit'] - df_summary['Debit']
#     return df_summary

# def create_bar_chart(df_summary, x_col, title):
#     """
#     Create a grouped bar chart comparing Debit and Credit.
#     """
#     fig = px.bar(
#         df_summary, 
#         x=x_col, 
#         y=['Debit', 'Credit'], 
#         text_auto='.2s',
#         labels={'value': 'Amount', 'variable': 'Type'}, 
#         barmode='group',
#         color_discrete_map={'Debit': 'lightcoral', 'Credit': 'lightblue'},
#         hover_data=['Total_Transactions', 'Balance']
#     )
#     fig.update_layout(
#         title=title, 
#         xaxis_title=x_col, 
#         yaxis_title='Amount', 
#         plot_bgcolor='whitesmoke'
#     )
#     return fig

# # ------------------------
# # Initialize Dash App with Bootstrap for Modals
# # ------------------------
# external_stylesheets = [dbc.themes.BOOTSTRAP]
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# # ------------------------
# # App Layout
# # ------------------------
# app.layout = html.Div([
#     html.H1("Comprehensive Transaction Analysis Dashboard", style={'textAlign': 'center'}),
#     dcc.Tabs([
#         # ----- Tab 1: General Analysis -----
#         dcc.Tab(label="General Analysis", children=[
#             html.Div([
#                 html.H2("General Transaction Analysis"),
#                 html.Label("Select Date Range"),
#                 dcc.DatePickerRange(
#                     id='date-picker-range',
#                     start_date=df['Transaction_Date'].min().date(),
#                     end_date=df['Transaction_Date'].max().date(),
#                     display_format='YYYY-MM-DD'
#                 ),
#                 # Grid layout: 2 charts per row (4 charts total)
#                 html.Div([
#                     # Frequency Distribution Chart Container
#                     html.Div([
#                         dcc.Graph(id='freq-bar', config={'displayModeBar': True}, style={'height': '300px'}),
#                         html.Button("Expand", id="btn-expand-freq", n_clicks=0, 
#                                     style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
#                     ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 'position': 'relative', 'margin': '10px'}),
#                     # Top Transactions Chart Container
#                     html.Div([
#                         dcc.Graph(id='top-transactions', config={'displayModeBar': True}, style={'height': '300px'}),
#                         html.Button("Expand", id="btn-expand-top", n_clicks=0, 
#                                     style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
#                     ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 'position': 'relative', 'margin': '10px'}),
#                     # Amount Distribution Chart Container
#                     html.Div([
#                         dcc.Graph(id='amount-pie', config={'displayModeBar': True}, style={'height': '300px'}),
#                         html.Button("Expand", id="btn-expand-amount", n_clicks=0, 
#                                     style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
#                     ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 'position': 'relative', 'margin': '10px'}),
#                     # Date Trend Chart Container
#                     html.Div([
#                         dcc.Graph(id='date-trend', config={'displayModeBar': True}, style={'height': '300px'}),
#                         html.Button("Expand", id="btn-expand-trend", n_clicks=0, 
#                                     style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
#                     ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 'position': 'relative', 'margin': '10px'}),
#                 ], style={'textAlign': 'center'})
#             ], style={'padding': '10px'}),
#             # Hidden Stores to hold figures for modals
#             dcc.Store(id='store-freq-bar'),
#             dcc.Store(id='store-top-transactions'),
#             dcc.Store(id='store-amount-pie'),
#             dcc.Store(id='store-date-trend'),
#             # Modals for Expanded Views (General Analysis)
#             dbc.Modal([
#                 dbc.ModalHeader("Frequency Distribution"),
#                 dbc.ModalBody(dcc.Graph(id='modal-freq-bar', style={'height': '600px'})),
#                 dbc.ModalFooter(dbc.Button("Close", id="close-modal-freq", className="ml-auto"))
#             ], id="modal-freq", size="xl", is_open=False),
#             dbc.Modal([
#                 dbc.ModalHeader("Top Transactions"),
#                 dbc.ModalBody(dcc.Graph(id='modal-top-transactions', style={'height': '600px'})),
#                 dbc.ModalFooter(dbc.Button("Close", id="close-modal-top", className="ml-auto"))
#             ], id="modal-top", size="xl", is_open=False),
#             dbc.Modal([
#                 dbc.ModalHeader("Transaction Amount Distribution"),
#                 dbc.ModalBody(dcc.Graph(id='modal-amount-pie', style={'height': '600px'})),
#                 dbc.ModalFooter(dbc.Button("Close", id="close-modal-amount", className="ml-auto"))
#             ], id="modal-amount", size="xl", is_open=False),
#             dbc.Modal([
#                 dbc.ModalHeader("Transaction Trends Over Time"),
#                 dbc.ModalBody(dcc.Graph(id='modal-date-trend', style={'height': '600px'})),
#                 dbc.ModalFooter(dbc.Button("Close", id="close-modal-trend", className="ml-auto"))
#             ], id="modal-trend", size="xl", is_open=False)
#         ]),
        
#         # ----- Tab 2: Breakdown Analysis -----
#         dcc.Tab(label="Breakdown Analysis", children=[
#             html.Div([
#                 html.H2("Transaction Breakdown Analysis"),
#                 html.Div([
#                     html.Label("Select Year"),
#                     dcc.Dropdown(
#                         id='year-dropdown',
#                         options=[{'label': str(y), 'value': y} for y in sorted(df['Year'].unique())],
#                         placeholder='Select Year'
#                     )
#                 ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top', 'margin': '10px'}),
#                 html.Div([
#                     html.Label("Select Month"),
#                     dcc.Dropdown(id='month-dropdown', placeholder='Select Month')
#                 ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top', 'margin': '10px'}),
#                 html.Div([
#                     html.Label("Select Week"),
#                     dcc.Dropdown(id='week-dropdown', placeholder='Select Week')
#                 ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top', 'margin': '10px'}),
#             ], style={'padding': '10px'}),
#             # Grid layout for Breakdown Analysis Charts (4 charts arranged in 2 rows)
#             html.Div([
#                 html.Div([
#                     dcc.Graph(id='yearly-summary', config={'displayModeBar': True}, style={'height': '300px'}),
#                     html.Button("Expand", id="btn-expand-yearly", n_clicks=0,
#                                 style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
#                 ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 'position': 'relative', 'margin': '10px'}),
#                 html.Div([
#                     dcc.Graph(id='monthly-summary', config={'displayModeBar': True}, style={'height': '300px'}),
#                     html.Button("Expand", id="btn-expand-monthly", n_clicks=0,
#                                 style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
#                 ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 'position': 'relative', 'margin': '10px'}),
#             ], style={'textAlign': 'center'}),
#             html.Div([
#                 html.Div([
#                     dcc.Graph(id='weekly-summary', config={'displayModeBar': True}, style={'height': '300px'}),
#                     html.Button("Expand", id="btn-expand-weekly", n_clicks=0,
#                                 style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
#                 ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 'position': 'relative', 'margin': '10px'}),
#                 html.Div([
#                     dcc.Graph(id='daily-summary', config={'displayModeBar': True}, style={'height': '300px'}),
#                     html.Button("Expand", id="btn-expand-daily", n_clicks=0,
#                                 style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
#                 ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 'position': 'relative', 'margin': '10px'}),
#             ], style={'textAlign': 'center'}),
#             # Hidden Stores for Breakdown Analysis modals
#             dcc.Store(id='store-yearly-summary'),
#             dcc.Store(id='store-monthly-summary'),
#             dcc.Store(id='store-weekly-summary'),
#             dcc.Store(id='store-daily-summary'),
#             # Modals for Expanded Breakdown Analysis Charts
#             dbc.Modal([
#                 dbc.ModalHeader("Yearly Summary"),
#                 dbc.ModalBody(dcc.Graph(id='modal-yearly-summary', style={'height': '600px'})),
#                 dbc.ModalFooter(dbc.Button("Close", id="close-modal-yearly", className="ml-auto"))
#             ], id="modal-yearly", size="xl", is_open=False),
#             dbc.Modal([
#                 dbc.ModalHeader("Monthly Summary"),
#                 dbc.ModalBody(dcc.Graph(id='modal-monthly-summary', style={'height': '600px'})),
#                 dbc.ModalFooter(dbc.Button("Close", id="close-modal-monthly", className="ml-auto"))
#             ], id="modal-monthly", size="xl", is_open=False),
#             dbc.Modal([
#                 dbc.ModalHeader("Weekly Summary"),
#                 dbc.ModalBody(dcc.Graph(id='modal-weekly-summary', style={'height': '600px'})),
#                 dbc.ModalFooter(dbc.Button("Close", id="close-modal-weekly", className="ml-auto"))
#             ], id="modal-weekly", size="xl", is_open=False),
#             dbc.Modal([
#                 dbc.ModalHeader("Daily Summary"),
#                 dbc.ModalBody(dcc.Graph(id='modal-daily-summary', style={'height': '600px'})),
#                 dbc.ModalFooter(dbc.Button("Close", id="close-modal-daily", className="ml-auto"))
#             ], id="modal-daily", size="xl", is_open=False),
#         ])
#     ])
# ])

# # ------------------------
# # Callbacks for General Analysis Tab
# # ------------------------
# @app.callback(
#     [Output('freq-bar', 'figure'),
#      Output('top-transactions', 'figure'),
#      Output('amount-pie', 'figure'),
#      Output('date-trend', 'figure'),
#      Output('store-freq-bar', 'data'),
#      Output('store-top-transactions', 'data'),
#      Output('store-amount-pie', 'data'),
#      Output('store-date-trend', 'data')],
#     [Input('date-picker-range', 'start_date'),
#      Input('date-picker-range', 'end_date')]
# )
# def update_general_analysis(start_date, end_date):
#     # Filter data based on selected date range
#     df_filtered = df[(df['Transaction_Date'] >= start_date) & (df['Transaction_Date'] <= end_date)].copy()
#     # Create Transaction_Key: Use Recipient_Name if available, otherwise use Transaction_ID
#     df_filtered['Transaction_Key'] = df_filtered.get('Recipient_Name', df_filtered['Transaction_ID'])
    
#     # Frequency Distribution
#     transaction_counts = df_filtered['Transaction_Key'].value_counts()
#     frequency_groups = {
#         '1 Transaction': sum(transaction_counts == 1),
#         '1-5 Transactions': sum((transaction_counts > 1) & (transaction_counts <= 5)),
#         '5-10 Transactions': sum((transaction_counts > 5) & (transaction_counts <= 10)),
#         '10-35 Transactions': sum((transaction_counts > 10) & (transaction_counts <= 35)),
#         '35-50 Transactions': sum((transaction_counts > 35) & (transaction_counts <= 50)),
#         'More than 50': sum(transaction_counts > 50)
#     }
#     freq_fig = px.bar(
#         x=list(frequency_groups.keys()),
#         y=list(frequency_groups.values()),
#         labels={'x': "Transaction Frequency", 'y': "Count"},
#         title="Transaction Frequency Distribution"
#     )
    
#     # Top 10 Frequent Transactions
#     top_transactions = transaction_counts.head(10)
#     top_fig = px.bar(
#         x=top_transactions.values,
#         y=top_transactions.index,
#         orientation='h',
#         labels={'x': "Transaction Count", 'y': "Transaction Key"},
#         title="Top 10 Frequent Transactions"
#     )
    
#     # Transaction Amount Distribution (based on Debit amounts)
#     transaction_ranges = {
#         '0-50': sum(df_filtered['Debit'] <= 50),
#         '50-100': sum((df_filtered['Debit'] > 50) & (df_filtered['Debit'] <= 100)),
#         '100-500': sum((df_filtered['Debit'] > 100) & (df_filtered['Debit'] <= 500)),
#         '500-1000': sum((df_filtered['Debit'] > 500) & (df_filtered['Debit'] <= 1000)),
#         '1000-5000': sum((df_filtered['Debit'] > 1000) & (df_filtered['Debit'] <= 5000)),
#         '5000-10000': sum((df_filtered['Debit'] > 5000) & (df_filtered['Debit'] <= 10000)),
#         '10000+': sum(df_filtered['Debit'] > 10000)
#     }
#     amount_pie = px.pie(
#         names=list(transaction_ranges.keys()),
#         values=list(transaction_ranges.values()),
#         title="Transaction Amount Distribution"
#     )
    
#     # Date-wise Transaction Trends (summing Debit amounts over time)
#     date_trend_df = df_filtered.groupby('Transaction_Date')['Debit'].sum().reset_index()
#     trend_fig = px.line(
#         date_trend_df,
#         x="Transaction_Date",
#         y="Debit",
#         title="Transaction Trends Over Time",
#         labels={'Transaction_Date': "Date", 'Debit': "Total Amount"}
#     )
    
#     # Return figures for small graphs and store copies for modals
#     return freq_fig, top_fig, amount_pie, trend_fig, freq_fig, top_fig, amount_pie, trend_fig

# # ------------------------
# # Callbacks for Breakdown Analysis Tab
# # ------------------------
# # Update Month dropdown based on selected Year
# @app.callback(
#     Output('month-dropdown', 'options'),
#     Input('year-dropdown', 'value')
# )
# def update_month_dropdown(selected_year):
#     if not selected_year:
#         return []
#     months = df[df['Year'] == selected_year][['Month_Num', 'Month']].drop_duplicates()
#     return [{'label': row['Month'], 'value': row['Month_Num']} for _, row in months.iterrows()]

# # Update Week dropdown based on selected Year and Month
# @app.callback(
#     Output('week-dropdown', 'options'),
#     [Input('year-dropdown', 'value'),
#      Input('month-dropdown', 'value')]
# )
# def update_week_dropdown(selected_year, selected_month):
#     if not selected_year or not selected_month:
#         return []
#     weeks = df[(df['Year'] == selected_year) & (df['Month_Num'] == selected_month)][['Week_Num']].drop_duplicates()
#     return [{'label': f'Week {w}', 'value': w} for w in sorted(weeks['Week_Num'])]

# # Yearly Summary: Group by Month within the selected Year
# @app.callback(
#     [Output('yearly-summary', 'figure'),
#      Output('store-yearly-summary', 'data')],
#     Input('year-dropdown', 'value')
# )
# def update_yearly_summary(selected_year):
#     if not selected_year:
#         return {}, {}
#     df_filtered = df[df['Year'] == selected_year]
#     df_summary = generate_summary(df_filtered, 'Month')
#     fig = create_bar_chart(df_summary, 'Month', f'Year {selected_year} Summary')
#     return fig, fig

# # Monthly Summary: Group by Week within the selected Year and Month
# @app.callback(
#     [Output('monthly-summary', 'figure'),
#      Output('store-monthly-summary', 'data')],
#     [Input('year-dropdown', 'value'),
#      Input('month-dropdown', 'value')]
# )
# def update_monthly_summary(selected_year, selected_month):
#     if not selected_year or not selected_month:
#         return {}, {}
#     df_filtered = df[(df['Year'] == selected_year) & (df['Month_Num'] == selected_month)]
#     df_summary = generate_summary(df_filtered, 'Week_Num')
#     df_summary['Week_Num'] = df_summary['Week_Num'].apply(lambda x: f'Week {x}')
#     fig = create_bar_chart(df_summary, 'Week_Num', f'{df_filtered["Month"].iloc[0]} Summary')
#     return fig, fig

# # Weekly Summary: Group by Day within the selected Year, Month, and Week
# @app.callback(
#     [Output('weekly-summary', 'figure'),
#      Output('store-weekly-summary', 'data')],
#     [Input('year-dropdown', 'value'),
#      Input('month-dropdown', 'value'),
#      Input('week-dropdown', 'value')]
# )
# def update_weekly_summary(selected_year, selected_month, selected_week):
#     if not selected_year or not selected_month or not selected_week:
#         return {}, {}
#     df_filtered = df[
#         (df['Year'] == selected_year) &
#         (df['Month_Num'] == selected_month) &
#         (df['Week_Num'] == selected_week)
#     ]
#     df_summary = generate_summary(df_filtered, 'Day')
#     fig = create_bar_chart(df_summary, 'Day', f'Week {selected_week} Summary')
#     return fig, fig

# # Daily Summary: Group by Day for detailed info in the selected Week
# @app.callback(
#     [Output('daily-summary', 'figure'),
#      Output('store-daily-summary', 'data')],
#     [Input('year-dropdown', 'value'),
#      Input('month-dropdown', 'value'),
#      Input('week-dropdown', 'value')]
# )
# def update_daily_summary(selected_year, selected_month, selected_week):
#     if not selected_year or not selected_month or not selected_week:
#         return {}, {}
#     df_filtered = df[
#         (df['Year'] == selected_year) &
#         (df['Month_Num'] == selected_month) &
#         (df['Week_Num'] == selected_week)
#     ]
#     df_summary = generate_summary(df_filtered, 'Day')
#     fig = create_bar_chart(df_summary, 'Day', f'Daily Summary for Week {selected_week}')
#     return fig, fig

# # ------------------------
# # Callbacks to Toggle General Analysis Modals
# # ------------------------
# @app.callback(
#     Output("modal-freq", "is_open"),
#     [Input("btn-expand-freq", "n_clicks"), Input("close-modal-freq", "n_clicks")],
#     [State("modal-freq", "is_open")]
# )
# def toggle_modal_freq(expand, close, is_open):
#     if expand or close:
#         return not is_open
#     return is_open

# @app.callback(
#     Output("modal-top", "is_open"),
#     [Input("btn-expand-top", "n_clicks"), Input("close-modal-top", "n_clicks")],
#     [State("modal-top", "is_open")]
# )
# def toggle_modal_top(expand, close, is_open):
#     if expand or close:
#         return not is_open
#     return is_open

# @app.callback(
#     Output("modal-amount", "is_open"),
#     [Input("btn-expand-amount", "n_clicks"), Input("close-modal-amount", "n_clicks")],
#     [State("modal-amount", "is_open")]
# )
# def toggle_modal_amount(expand, close, is_open):
#     if expand or close:
#         return not is_open
#     return is_open

# @app.callback(
#     Output("modal-trend", "is_open"),
#     [Input("btn-expand-trend", "n_clicks"), Input("close-modal-trend", "n_clicks")],
#     [State("modal-trend", "is_open")]
# )
# def toggle_modal_trend(expand, close, is_open):
#     if expand or close:
#         return not is_open
#     return is_open

# # ------------------------
# # Callbacks to Toggle Breakdown Analysis Modals
# # ------------------------
# @app.callback(
#     Output("modal-yearly", "is_open"),
#     [Input("btn-expand-yearly", "n_clicks"), Input("close-modal-yearly", "n_clicks")],
#     [State("modal-yearly", "is_open")]
# )
# def toggle_modal_yearly(expand, close, is_open):
#     if expand or close:
#         return not is_open
#     return is_open

# @app.callback(
#     Output("modal-monthly", "is_open"),
#     [Input("btn-expand-monthly", "n_clicks"), Input("close-modal-monthly", "n_clicks")],
#     [State("modal-monthly", "is_open")]
# )
# def toggle_modal_monthly(expand, close, is_open):
#     if expand or close:
#         return not is_open
#     return is_open

# @app.callback(
#     Output("modal-weekly", "is_open"),
#     [Input("btn-expand-weekly", "n_clicks"), Input("close-modal-weekly", "n_clicks")],
#     [State("modal-weekly", "is_open")]
# )
# def toggle_modal_weekly(expand, close, is_open):
#     if expand or close:
#         return not is_open
#     return is_open

# @app.callback(
#     Output("modal-daily", "is_open"),
#     [Input("btn-expand-daily", "n_clicks"), Input("close-modal-daily", "n_clicks")],
#     [State("modal-daily", "is_open")]
# )
# def toggle_modal_daily(expand, close, is_open):
#     if expand or close:
#         return not is_open
#     return is_open

# # ------------------------
# # Callbacks to Update Modal Graphs (General Analysis)
# # ------------------------
# @app.callback(
#     Output("modal-freq-bar", "figure"),
#     Input("store-freq-bar", "data")
# )
# def update_modal_freq_bar(store_data):
#     return store_data

# @app.callback(
#     Output("modal-top-transactions", "figure"),
#     Input("store-top-transactions", "data")
# )
# def update_modal_top(store_data):
#     return store_data

# @app.callback(
#     Output("modal-amount-pie", "figure"),
#     Input("store-amount-pie", "data")
# )
# def update_modal_amount(store_data):
#     return store_data

# @app.callback(
#     Output("modal-date-trend", "figure"),
#     Input("store-date-trend", "data")
# )
# def update_modal_trend(store_data):
#     return store_data

# # ------------------------
# # Callbacks to Update Modal Graphs (Breakdown Analysis)
# # ------------------------
# @app.callback(
#     Output("modal-yearly-summary", "figure"),
#     Input("store-yearly-summary", "data")
# )
# def update_modal_yearly_summary(store_data):
#     return store_data

# @app.callback(
#     Output("modal-monthly-summary", "figure"),
#     Input("store-monthly-summary", "data")
# )
# def update_modal_monthly_summary(store_data):
#     return store_data

# @app.callback(
#     Output("modal-weekly-summary", "figure"),
#     Input("store-weekly-summary", "data")
# )
# def update_modal_weekly_summary(store_data):
#     return store_data

# @app.callback(
#     Output("modal-daily-summary", "figure"),
#     Input("store-daily-summary", "data")
# )
# def update_modal_daily_summary(store_data):
#     return store_data

# # ------------------------
# # Run the Dash App
# # ------------------------
# if __name__ == '__main__':
#     app.run_server(debug=True, port=8060)


In [29]:
def print_transaction_details(df):
    """
    Prints every row with all the extracted details from the DataFrame.
    """
    for index, row in df.iterrows():
        print(f"Row {index}:")
        for col in df.columns:
            print(f"  {col}: {row[col]}")
        print("-" * 50)  # Separator for better readability
# print_transaction_details(daily_counts)
# print_transaction_details(df)
        
